<a href="https://colab.research.google.com/github/karavdin/DeepPiCar/blob/master/models/object_detection/code/tensorflow_traffic_sign_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Raspberry Pi to Detect Traffic Signs and People in Real Time

This is tutorial is based on Chengwei's excellent Tutorial and Colab Notebook on ["How to train an object detection model easy for free"](https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/).   My twist on his tutorial is that I need to run my model on a Raspberry Pi with live video feed.  As the Raspberry Pi is fairly limited on CPU power and can only run object detection at 1-2 FPS (frames/sec), I have purchased the newly release $75 Google's [EdgeTPU USB Accelarator](https://coral.withgoogle.com/products/accelerator), which can detect objects at 12 FPS, which is sufficient for real time work.  After doing the transfer learning from one of the object detection models using our own images, last few steps of the colab deals with how to convert a trained model to a model file that can be consumed by an Edge TPU, namely, the final `mymodel_quantized_edgetpu.tflite` file.  


![](https://cdn-images-1.medium.com/max/1000/1*_jABdMfUVcyPdi5b3zlfVg.jpeg)



# Section 1: Mount Google drive
Mount my Google Drive and save modeling output files (`.ckpt`)  there, so that it won't be wiped out when colab Virtual Machine restarts.  It has an idle timeout of 90 min, and maximum daily usage of 12 hours.

Google will ask for an authenticate code when you run the following code, just follow the link in the output and allow access.   You can put the `model_dir` anywhere in your google drive.

In [1]:

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
model_dir = '/content/gdrive/My Drive/Colab Notebooks/DeepCar_Training'
#!rm -rf '{model_dir}'
#os.makedirs(model_dir, exist_ok=True)
!ls -ltra '{model_dir}'/..

Mounted at /content/gdrive
total 27181
-rw------- 1 root root   79210 Jun 30  2019  my_em_assignment.ipynb
-rw------- 1 root root  395675 Jul  9  2019 'my mcmc_assignment.ipynb'
-rw------- 1 root root  206381 Jul 24  2019  my_Vae_assignment.ipynb
-rw------- 1 root root  243880 Jul 28  2019  my_gp_assignment.ipynb
-rw------- 1 root root  968824 Jul 29  2019  my_finding_suspect.ipynb
-rw------- 1 root root     287 Aug 22  2019  Untitled0.ipynb
-rw------- 1 root root 1698449 Aug 22  2019 'Bike Sharing Prediction.ipynb'
-rw------- 1 root root     287 Aug 27  2019  Untitled1.ipynb
-rw------- 1 root root  781572 Aug 28  2019 'Trip waiting time prediction.ipynb'
-rw------- 1 root root   19520 Sep 10  2019 'Data Engineer Challenge Swarovski.ipynb'
-rw------- 1 root root     269 Sep 22  2019  Untitled2.ipynb
-rw------- 1 root root  190302 Sep 24  2019  Prototype2.ipynb
-rw------- 1 root root  128216 Oct 30  2019 'My Copy of Exercise_Generative_Models_Problem.ipynb'
-rw------- 1 root root  26892

# Section 2: Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [2]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/karavdin/DeepPiCar'

# Number of training steps.
#num_steps = 1000  # 200000
num_steps = 100  # 200000

# Number of evaluation steps.
#num_eval_steps = 50
num_eval_steps = 5


# model configs are from Model Zoo github: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
MODELS_CONFIG = {
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
    'ssd_mobilenet_v1_quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18',
        'pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'batch_size': 12
    },    
    # 'ssd_mobilenet_v2': {
    #     'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    #     'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    #     'batch_size': 12
    # },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
    'ssd_mobilenet_v2_tf2': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        #'pipeline_file': 'ssd_mobilenet_v3_small.config',
        'batch_size': 12
    },
    
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
# Note: for Edge TPU, you have to:
# 1) start with a pretrained model from model zoo, such as above 4
# 2) Must be a quantized model, which reduces the model size significantly
#selected_model = 'ssd_mobilenet_v2_quantized'
selected_model = 'ssd_mobilenet_v2_tf2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
#pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

# Section 3: Set up Training Environment

## Clone the `DeepPiCar` repository or your fork.

In [3]:
import os
import pandas as pd

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}

print('Pull it so that we have the latest code/data')
!git pull

/content
Cloning into 'DeepPiCar'...
remote: Enumerating objects: 1334, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 1334 (delta 158), reused 206 (delta 109), pack-reused 1027
Receiving objects: 100% (1334/1334), 184.37 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (479/479), done.
/content/DeepPiCar
Pull it so that we have the latest code/data
Already up to date.


## Install required packages

In [4]:

#!pip uninstall tensorflow

In [5]:
%cd /content

# !pip install tf_slim
# !pip install tensorflow_io
# !pip install tensorflow-addons
!pip install tensorflow==2.11.0
#!pip install --ignore-installed --upgrade tensorflow==1.15

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 34.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorboard
    Found e

In [6]:
import tensorflow as tf
print("--- Tensorflow version --- \n", tf.__version__)
#print(tf.reduce_sum(tf.random.normal([1000, 1000])))

--- Tensorflow version --- 
 2.11.0


In [7]:
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q cython contextlib2 pillow lxml matplotlib
#!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI


# !git clone https://github.com/cocodataset/cocoapi.git
# %cd cocoapi/PythonAPI
# !make
# !cp -r pycocotools /content/models/research/

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!python -m pip install --use-feature=2020-resolver .

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/:/content/models/'

import sys
sys.path.append("/content/models")

#!python object_detection/builders/model_builder_test.py

Selecting previously unselected package python2-minimal.
(Reading database ... 129499 files and directories currently installed.)
Preparing to unpack .../python2-minimal_2.7.17-2ubuntu4_amd64.deb ...
Unpacking python2-minimal (2.7.17-2ubuntu4) ...
Selecting previously unselected package libpython2-stdlib:amd64.
Preparing to unpack .../libpython2-stdlib_2.7.17-2ubuntu4_amd64.deb ...
Unpacking libpython2-stdlib:amd64 (2.7.17-2ubuntu4) ...
Setting up python2-minimal (2.7.17-2ubuntu4) ...
Selecting previously unselected package python2.
(Reading database ... 129528 files and directories currently installed.)
Preparing to unpack .../00-python2_2.7.17-2ubuntu4_amd64.deb ...
Unpacking python2 (2.7.17-2ubuntu4) ...
Selecting previously unselected package libimagequant0:amd64.
Preparing to unpack .../01-libimagequant0_2.12.2-1.1_amd64.deb ...
Unpacking libimagequant0:amd64 (2.12.2-1.1) ...
Selecting previously unselected package python-backports.functools-lru-cache.
Preparing to unpack .../02-p

In [8]:
!python object_detection/builders/model_builder_tf2_test.py

2023-01-28 14:46:45.790171: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:46:45.790342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:46:45.790371: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-28 14:46:50.966980: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

## Prepare `tfrecord` files

Use the following scripts to generate the `tfrecord` files.

```

In [9]:
%cd {repo_dir_path}/models/object_detection

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python code/xml_to_csv.py -i data/images_hh/train -o data/annotations_hh/train_labels.csv -l data/annotations_hh

# Convert test folder annotation xml files to a single csv.
!python code/xml_to_csv.py -i data/images_hh/test -o data/annotations_hh/test_labels.csv



/content/DeepPiCar/models/object_detection
Successfully converted xml to csv.
Generate `data/annotations_hh/label_map.pbtxt`
Successfully converted xml to csv.


In [10]:
!ls

code  data  docs


In [11]:
df_train_labels = pd.read_csv('data/annotations_hh/train_labels.csv')
print(df_train_labels['class'].unique())

['Capitan' 'Nastya' 'Cam' 'Sheila' 'Bob' 'bike' 'Babushka'
 'e-bike station' 'Phil' 'clock' 'ukulele' 'disko' 'Ukulele']


In [12]:
df_test_labels = pd.read_csv('data/annotations_hh/test_labels.csv')
print(df_test_labels['class'].unique())

['clock' 'bike' 'Babushka' 'Nastya' 'e-bike station' 'Sheila' 'Capitan'
 'Bob' 'Phil' 'Cam' 'ukulele']


In [13]:
# Generate `train.record`
!python code/generate_tfrecord.py --csv_input=data/annotations_hh/train_labels.csv --output_path=data/annotations_hh/train.record --img_path=data/images_hh/train --label_map data/annotations_hh/label_map.pbtxt

# Generate `test.record`
!python code/generate_tfrecord.py --csv_input=data/annotations_hh/test_labels.csv --output_path=data/annotations_hh/test.record --img_path=data/images_hh/test --label_map data/annotations_hh/label_map.pbtxt

2023-01-28 14:48:01.884101: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:48:01.884305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:48:01.884333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-28 14:48:05.072906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

In [14]:
test_record_fname = repo_dir_path + '/models/object_detection/data/annotations_hh/test.record'
train_record_fname = repo_dir_path + '/models/object_detection/data/annotations_hh/train.record'
label_map_pbtxt_fname = repo_dir_path + '/models/object_detection/data/annotations_hh/label_map.pbtxt'

In [15]:
!cat data/annotations_hh/test_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
test_37.jpg,2592,1944,clock,10,454,304,1114
test_37.jpg,2592,1944,bike,121,948,299,1270
test_37.jpg,2592,1944,Babushka,675,954,824,1250
test_37.jpg,2592,1944,Nastya,812,974,870,1123
test_37.jpg,2592,1944,e-bike station,1264,663,1639,1290
test_37.jpg,2592,1944,Sheila,1712,1028,1786,1197
test_37.jpg,2592,1944,Capitan,1924,970,2221,1514
test_37.jpg,2592,1944,Bob,2272,1034,2448,1341
test_37.jpg,2592,1944,Phil,2019,746,2239,994
test_2.jpg,2592,1944,Bob,2357,1019,2592,1399
test_2.jpg,2592,1944,Phil,2099,694,2348,1219
test_2.jpg,2592,1944,Capitan,1908,1012,2041,1237
test_2.jpg,2592,1944,clock,1804,803,2044,1150
test_2.jpg,2592,1944,Babushka,1495,992,1637,1146
test_2.jpg,2592,1944,Cam,1312,994,1404,1112
test_2.jpg,2592,1944,Sheila,1052,963,1161,1123
test_2.jpg,2592,1944,Nastya,670,959,768,1159
test_2.jpg,2592,1944,ukulele,695,1026,752,1128
test_2.jpg,2592,1944,e-bike station,237,452,666,1294
test_28.jpg,2592,1944,Nastya,715,952,837,1197
test_28.j

## Download base model

In [16]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
#DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
print(MODEL,"from",MODEL_FILE)

/content/models/research
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8 from ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz


In [17]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 24K
drwxr-x---  4 345018 89939 4.0K Jul 11  2020 .
drwxr-xr-x 25 root   root  4.0K Jan 28 14:48 ..
drwxr-x---  2 345018 89939 4.0K Jul 10  2020 checkpoint
-rw-r-----  1 345018 89939 4.5K Jul 11  2020 pipeline.config
drwxr-x---  3 345018 89939 4.0K Jul 10  2020 saved_model


In [18]:
fine_tune_checkpoint = os.path.join(DEST_DIR+"/checkpoint", "ckpt-0")
fine_tune_checkpoint

'/content/models/research/pretrained_model/checkpoint/ckpt-0'

# Section 4: Transfer Learning Training

## Configuring a Training Pipeline

In [19]:
import os
pipeline_fname = os.path.join(DEST_DIR, 'pipeline.config')
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [20]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [21]:
import re

# training pipeline file defines:
# - pretrain model path
# - the train/test sets
# - ID to Label mapping and number of classes
# - training batch size
# - epochs to trains
# - learning rate
# - etc

# note we just need to use a sample one, and make edits to it.

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    s = s = re.sub(
        '(fine_tune_checkpoint_type: ".*?)(classification)(.*?")', 'fine_tune_checkpoint_type: "{}"'.format("detection"), s, 1)

    # fine_tune_checkpoint: downloaded pre-trained model checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test, we created earlier with our training/test sets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 'input_path: "{}"'.format(train_record_fname), s, 1)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 'input_path: "{}"'.format(test_record_fname), s, 1)

    # label_map_path: ID to label file
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps (Number of epochs to train)
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
  
    f.write(s)

In [22]:
#!cat {label_map_pbtxt_fname}

In [23]:
# look for num_classes: 13, since we have 13 different objects 
!cat {pipeline_fname}

model {
  ssd {
    num_classes: 13
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0


## Run Tensorboard(Optional)

In [24]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip -o ngrok-stable-linux-amd64.zip

--2023-01-28 14:48:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  58.4MB/s    in 0.2s    

2023-01-28 14:48:18 (58.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [25]:
# LOG_DIR = model_dir
# get_ipython().system_raw(
#     'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
#     .format(LOG_DIR)
# )

In [26]:
#get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [27]:
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


## Train the model

Now all inputs are set up, just train the model.   This process may take a few hours.   Since we are saving the model training results (model.ckpt-* files) in our google drive (a persistent storage that will survice the restart of our colab VM instance), we can safely leave and return a few hours later. 

In [29]:
!pip install lvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
num_steps = 20
#SendEmail("Colab train started")
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir='{model_dir}' \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}
#SendEmail("Colab train finished")

2023-01-28 14:48:32.474609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:48:32.474767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 14:48:32.474790: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-28 14:48:37.669072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

In [31]:
!ls -ltra '{model_dir}'

total 30898
drwx------ 2 root root     4096 Jan 22 12:16 fine_tuned_model
-rw------- 1 root root    26246 Jan 24 06:49 ckpt-1.index
-rw------- 1 root root 10516881 Jan 24 06:49 ckpt-1.data-00000-of-00001
-rw------- 1 root root    26246 Jan 24 06:53 ckpt-2.index
-rw------- 1 root root 10516881 Jan 24 06:53 ckpt-2.data-00000-of-00001
drwx------ 2 root root     4096 Jan 28 14:49 train
-rw------- 1 root root 10516881 Jan 28 14:49 ckpt-3.data-00000-of-00001
-rw------- 1 root root    26246 Jan 28 14:49 ckpt-3.index
-rw------- 1 root root      343 Jan 28 14:49 checkpoint


# Section 5: Save and Convert Model Output

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [32]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

In [34]:
# lst = os.listdir(model_dir)
# # find the last model checkpoint file, i.e. model.ckpt-1000.meta
# lst = [l for l in lst if 'ckpt-' in l and '.index' in l]
# #print(lst)
# steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
# last_model = lst[steps.argmax()].replace('.index', '')
# #last_model = lst[steps.argmax()]
# last_model_path = os.path.join(model_dir, last_model)
# print(last_model_path)


/content/gdrive/My Drive/Colab Notebooks/DeepCar_Training/ckpt-3


In [35]:
!ls /content/models/research/object_detection/

anchor_generators			 __init__.py
box_coders				 inputs.py
builders				 inputs_test.py
colab_tutorials				 legacy
configs					 matchers
CONTRIBUTING.md				 meta_architectures
core					 metrics
data					 model_hparams.py
data_decoders				 model_lib.py
dataset_tools				 model_lib_tf1_test.py
dockerfiles				 model_lib_tf2_test.py
eval_util.py				 model_lib_v2.py
eval_util_test.py			 model_main.py
exporter_lib_tf2_test.py		 model_main_tf2.py
exporter_lib_v2.py			 models
exporter_main_v2.py			 model_tpu_main.py
exporter.py				 packages
exporter_tf1_test.py			 predictors
export_inference_graph.py		 protos
export_tflite_graph_lib_tf2.py		 __pycache__
export_tflite_graph_lib_tf2_test.py	 README.md
export_tflite_graph_tf2.py		 samples
export_tflite_ssd_graph_lib.py		 test_data
export_tflite_ssd_graph_lib_tf1_test.py  test_images
export_tflite_ssd_graph.py		 tpu_exporters
g3doc					 utils
inference


In [42]:
!echo exporting the model
# there is an "Incomplete shape" message.  but we can safely ignore that. 
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --output_directory='{output_directory}' \
    --trained_checkpoint_dir='{model_dir}'

creates the frozen inference graph in fine_tune_model
2023-01-28 15:43:15.812867: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 15:43:15.813003: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 15:43:15.813021: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-28 15:43:19.289868: E tensorflow/compiler/xla/str

In [36]:
# !echo creates the frozen inference graph in fine_tune_model
# # there is an "Incomplete shape" message.  but we can safely ignore that. 
# !python /content/models/research/object_detection/export_tflite_graph_tf2.py \
#     --input_type=image_tensor \
#     --pipeline_config_path={pipeline_fname} \
#     --output_directory='{output_directory}' \
#     --trained_checkpoint_prefix='{last_model_path}'

creates the frozen inference graph in fine_tune_model
2023-01-28 15:02:51.707089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 15:02:51.707441: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-28 15:02:51.707484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
FATAL Flags parsing error: Unknown command line flag 'inp

In [52]:
!echo convert saved model to TF-Lite
#https://www.tensorflow.org/lite/models/convert/convert_models#convert_a_savedmodel_recommended_
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(f'{output_directory}/saved_model/') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open(f'{output_directory}/saved_model/model.tflite', 'wb') as f:
  f.write(tflite_model)


convert saved model to TF-Lite


In [57]:
!ls -lh /content/gdrive/My\ Drive/Colab\ Notebooks/DeepCar_Training/fine_tuned_model/saved_model/

total 17M
drwx------ 2 root root 4.0K Jan 28 15:44 assets
-rw------- 1 root root   55 Jan 28 15:44 fingerprint.pb
-rw------- 1 root root  10M Jan 28 16:00 model.tflite
-rw------- 1 root root 6.5M Jan 28 15:44 saved_model.pb
drwx------ 2 root root 4.0K Jan 28 15:44 variables


## Run inference test
Test with images in repository `object_detection/data/images/test` directory.

In [85]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "models/object_detection/data/images_hh/test/")

In [86]:
PATH_TO_TEST_IMAGES_DIR

'/content/DeepPiCar/models/object_detection/data/images_hh/test/'

In [88]:
!ls /content/DeepPiCar/models/object_detection/data/images_hh/test/

test_10.jpg	    test_23.jpg		test_37.jpg	    test_47.jpg
test_10_labels.xml  test_23_labels.xml	test_37_labels.xml  test_47_labels.xml
test_17.jpg	    test_28.jpg		test_41.jpg	    test_6.jpg
test_17_lables.xml  test_28_labels.xml	test_41_labels.xml  test_6_labels.xml
test_22.jpg	    test_2.jpg		test_44.jpg
test_22_labels.xml  test_2_labels.xml	test_44_labels.xml


In [99]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
# Load saved model and build the detection function
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(f'{output_directory}/saved_model/')
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 15.081169366836548 seconds


In [103]:
print(list(detect_fn.signatures.keys()))
infer = detect_fn.signatures["serving_default"]


['serving_default']


In [97]:
print(detect_fn)

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f7f1f4b8070>


In [71]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [101]:
?detect_fn

In [126]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

directory = os.fsencode(PATH_TO_TEST_IMAGES_DIR)
for input_file in os.listdir(directory):
  filename = os.fsdecode(input_file)
  if filename.endswith(".jpg"):
    
    print('Running inference for {}... '.format(filename), end='')
    image_np = load_image_into_numpy_array(PATH_TO_TEST_IMAGES_DIR+'/'+filename)

    converted_img  = tf.image.convert_image_dtype(image_np, tf.float32)[tf.newaxis, ...]
    detections = infer(converted_img)
 
    print(detections)

    detections = {key:value.numpy() for key,value in detections.items()}
    print("Found %d objects." % len(detections["detection_scores"]))
    image_with_boxes = draw_boxes(
      image_np, detections["detection_boxes"],
      detections["detection_class_entities"], detections["detection_scores"])
    display_image(image_with_boxes)


    # # Things to try:
    # # Flip horizontally
    # # image_np = np.fliplr(image_np).copy()

    # # Convert image to grayscale
    # # image_np = np.tile(
    # #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    # input_tensor = tf.convert_to_tensor(image_np)
    # # The model expects a batch of images, so add an axis with `tf.newaxis`.
    # input_tensor = input_tensor[tf.newaxis, ...]
    # #print(input_tensor)
    # #input_tensor = np.expand_dims(image_np, 0)
    # #detections = infer(input_tensor)
    # detections = infer(tf.constant(input_tensor))['output']


    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()


Running inference for test_6.jpg... {'output_0': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'output_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'output_3': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'output_2': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>}


KeyError: ignored

In [117]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = f'{output_directory}/saved_model/'
pb_fname = f'{PATH_TO_CKPT}/saved_model.pb'
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "models/object_detection/data/images_hh/test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

/content/gdrive/My Drive/Colab Notebooks/DeepCar_Training/fine_tuned_model/saved_model/
['/content/DeepPiCar/models/object_detection/data/images_hh/test/test_6.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_37.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_28.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_47.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_22.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_17.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_10.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_2.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_23.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_41.jpg', '/content/DeepPiCar/models/object_detection/data/images_hh/test/test_44.jpg']


In [119]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(pb_fname, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict



/content/models/research/object_detection


DecodeError: ignored

In [ ]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

## Convert to Edge TPU's tflite Format  
The only known way, at time of writing (April 2019), is to download the below quantized tflite file from above, and use [Google's web compiler](https://coral.withgoogle.com/web-compiler/) to convert to Edge TPU's tflite format.   Unfortunately, this step has to be done by hand, and NOT via a script.  

Here are the requirements of Edge TPU web compiler.  If you have followed the above steps closely, you have met these requirements.

- Tensor parameters are quantized (8-bit fixed-point numbers). You must use quantization-aware training (post-training quantization is not supported).   (this is why we are using `ssd_mobilenet_v2_quantized` base model and not the  `ssd_mobilenet_v2` base model   
- Tensor sizes are constant at compile-time (no dynamic sizes).
- Model parameters (such as bias tensors) are constant at compile-time.
- Tensors are either 1-, 2-, or 3-dimensional. If a tensor has more than 3 dimensions, then only the 3 innermost dimensions may have a size greater than 1.
- The model uses only the operations supported by the Edge TPU 

In [ ]:
# download this file from google drive.
!ls -lt '/content/gdrive/My Drive/Colab Notebooks/TransferLearning/Training/fine_tuned_model/road_signs_quantized.tflite'

Wait for about 1-2 minutes for compilation to finish.  And we can download the model file as `road_signs_quantized_edgetpu.tflite`.  This is the file you need to copy to raspberry pi with TPU to run object detection.

We are all done with colab notebook training, now time to switch back to raspberry pi, and run `~/DeepPiCar/models/object_detection/code/object_detection_usb.py`.  You should see a video feed where road sign and persons are boxed with confidence level around them.  

```bash
# make sure the the road_signs_quantized_edgetpu.tflite is in the right directory in your pi
pi@raspberrypi:~/DeepPiCar/models/object_detection/data/model_result $ ls -ltr
total 10040
-rw-r--r-- 1 pi pi      97 Apr 15 01:01 road_sign_labels.txt
-rw-r--r-- 1 pi pi 4793504 Apr 16 15:49 road_signs_quantized.tflite
-rw-r--r-- 1 pi pi 5478080 Apr 16 15:49 road_signs_quantized_edgetpu.tflite

pi@raspberrypi:~/DeepPiCar/models/object_detection $ python3 code/object_detection_usb.py

------
2019-04-16 16:22:28.489224: 13.49 FPS, 74.12ms total, 70.84ms in tf 
Green Traffic Light, 80% [[240.61578751 131.68985367]
 [287.21975327 195.79172134]] 60.42ms
Stop Sign, 44% [[  0.         305.1651001 ]
 [180.84949493 409.32563782]] 60.42ms

------
2019-04-16 16:22:28.618309: 14.83 FPS, 67.44ms total, 60.42ms in tf 
Person, 89% [[505.6583786  279.52325821]
 [530.85933685 360.0169754 ]] 62.54ms
Green Traffic Light, 72% [[237.96649933 130.58757782]
 [283.52127075 203.24180603]] 62.54ms
Red Traffic Light, 62% [[283.23583603 169.27398682]
 [330.91316223 269.20692444]] 62.54ms
Stop Sign, 56% [[ 51.01628304 165.80377579]
 [101.48646355 227.05183029]] 62.54ms
Person, 44% [[396.8661499  298.65327835]
 [468.4034729  422.04421997]] 62.54ms
------

```

# Section 6: Last Words on this Project

Of course, depending many factors, not all objects in the video frame will be identified.  This is the chance to improve your model.   Try to train longer, or train with more labeled images, or augment your existing images with different zooms/rotations/contrast/lighting.  In my case, I started with one camera, which was somewhat fuzzy, and precision low.  When I switched to a HD camera, the model precision was significantly better.   This was just another way.

It is awesome that for just over $100 in hardware, we can do real time object detection at home.   Moreover, thanks to Google, all you need is a browser to train this huge model!  Having fun with your own raspberry pi object detection projects!  